In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 1. 다나와 데이터 수집 1페이지 크롤링

In [2]:
url = 'https://www.danawa.com/'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(2)

In [3]:
driver.find_element(By.CSS_SELECTOR, '.search__input').send_keys('무선청소기')
driver.find_element(By.CSS_SELECTOR, '.search__submit').click()

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

## 다나와 검색 웹페이지에서 상품정보 가져오기


In [6]:
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

44

In [7]:
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
title

'삼성전자 비스포크 제트 VS20A956A3'

In [8]:
# 스펙 정보 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t', "").replace('\n', '')
spec_list

'핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm'

In [11]:
# 가격 정보 가져오기
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
price

'458920'

In [10]:
# 1페이지
prod_data = []
for prod_item in prod_items:
    # 공백 ('li.prod_item.product-pot') 제외
    if 'product-pot' in prod_item['class']:
        continue
    try:
        title = prod_items[0].select('p.prod_name > a')[0].text.strip()
        spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t', "").replace('\n', '')
        price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
        prod_data.append([title, spec_list, price])
    except:
        pass
print(len(prod_data))
print(prod_data)

43
[['삼성전자 비스포크 제트 VS20A956A3', '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm', '458920'], ['삼성전자 비스포크 제트 VS20A956A3', '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm', '458920'], ['삼성전자 비스포크 제트 VS20A956A3', '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔형 / 틈새 / 연장툴 / 청정스테이션 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간(개당): 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] 물걸레(별도구매) / 디스플레이표시창 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 물통용량: 0.15L / 색상: 미드나잇블루 / 무게: 2.5kg / 액세서리크래들  / 크기(가로x세로x깊이): 250x930x202mm',

In [14]:
# 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        # 공백 ('li.prod_item.product-pot') 제외
        if 'product-pot' in prod_item['class']:
            continue
        try:
            title = prod_items[0].select('p.prod_name > a')[0].text.strip()
            spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t', "").replace('\n', '')
            price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
            prod_data.append([title, spec_list, price])
        except:
            pass
    return prod_data

In [15]:
# 상품 정보를 가져오는 함수 테스트
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

43


# 2. 여러 페이지에 걸친 다나와 검색 페이지 크롤링

In [16]:
# 다나와 검색 URL을 만들어 주는 함수
def get_search_page_url(keyword, page):
    return f'https://search.danawa.com/dsearch.php?query={keyword}&originalQuery={keyword}&previousKeyword={keyword}&volumeType=allvs&page={page}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2942&defaultVaTab=719721&tab=goods'

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&volumeType=allvs&page=1&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&recommendedSort=Y&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=2942&defaultVaTab=719721&tab=goods


In [18]:
import time
from tqdm.notebook import tqdm

In [22]:
keyword = '무선청소기'
total_page = 10
prod_data_total = []
for page in tqdm(range(1, total_page+1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 상품정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    
    # 추출한 데이터 저장
    prod_data_total = prod_data_total + prod_item_list

  0%|          | 0/10 [00:00<?, ?it/s]

# 3. 수집한 데이터 저장

In [24]:
df = pd.DataFrame(prod_data_total)
df.columns = ['상품명', '스펙 목록', '가격']
df.to_csv('danawa_crawling_result.csv', index=False)